In [ ]:
#! python --version

In [ ]:
#! pip3 install gym

In [ ]:
#! pip3 install pygame

# Create the environment

In [3]:
import gym

In [2]:
env = gym.make("CartPole-v1", render_mode="human")

In [4]:
env.reset()

(array([-0.04511656,  0.00330043,  0.01300843,  0.00355254], dtype=float32),
 {})

In [ ]:
# neutral state

#env.reset()

In [5]:
env.close()

In [6]:
env.action_space

Discrete(2)

In [12]:
env.action_space.sample()

1

In [18]:
env = gym.make("CartPole-v1", render_mode="human")

episodes = 50

for e in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample() # we are taking random actions, left and right
        n_state, reward, done, _, info = env.step(action)
        score += reward
    print(f"Episode #: {e}")
    print(f"Score: {score}")

env.close()

Episode #: 1
Score: 20.0
Episode #: 2
Score: 40.0
Episode #: 3
Score: 23.0
Episode #: 4
Score: 20.0
Episode #: 5
Score: 16.0
Episode #: 6
Score: 17.0
Episode #: 7
Score: 16.0
Episode #: 8
Score: 19.0
Episode #: 9
Score: 16.0
Episode #: 10
Score: 20.0
Episode #: 11
Score: 24.0
Episode #: 12
Score: 24.0
Episode #: 13
Score: 18.0
Episode #: 14
Score: 12.0
Episode #: 15
Score: 18.0
Episode #: 16
Score: 9.0
Episode #: 17
Score: 21.0
Episode #: 18
Score: 14.0
Episode #: 19
Score: 32.0
Episode #: 20
Score: 11.0
Episode #: 21
Score: 12.0
Episode #: 22
Score: 29.0
Episode #: 23
Score: 19.0
Episode #: 24
Score: 16.0
Episode #: 25
Score: 24.0
Episode #: 26
Score: 14.0
Episode #: 27
Score: 17.0
Episode #: 28
Score: 15.0
Episode #: 29
Score: 49.0
Episode #: 30
Score: 36.0
Episode #: 31
Score: 20.0
Episode #: 32
Score: 13.0
Episode #: 33
Score: 39.0


KeyboardInterrupt: 

In [ ]:
env.close()

## Train the agent

In [ ]:
!pip3 install 'stable-baselines3[extra]'

In [4]:
from stable_baselines3 import PPO # the algorithm we are importing
# reference: https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html
from stable_baselines3.common.vec_env import DummyVecEnv
# vectorise environment - train on multiple agents
# wrapper around environment
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
PPO?

In [5]:
# create our environment again
env = gym.make("CartPole-v1", render_mode="human")
# vectorise our environment
env = DummyVecEnv([lambda:env])
# import the algorithm
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log="training/logs")

/opt/miniconda3/envs/rl/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device


In [19]:
model.learn(total_timesteps=2000)

Logging to training/logs/PPO_5
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 500      |
|    ep_rew_mean     | 500      |
| time/              |          |
|    fps             | 47       |
|    iterations      | 1        |
|    time_elapsed    | 42       |
|    total_timesteps | 2048     |
---------------------------------


In [20]:
model_path = "training/models/cartpole"
model.save(model_path)

In [22]:
del model

NameError: name 'model' is not defined

## Evaluate the Model

In [23]:
# import model
env = gym.make("CartPole-v1", render_mode="human")
model_path = "training/models/cartpole"
model = PPO.load(model_path, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [24]:
evaluate_policy(model, env, n_eval_episodes=5, render=True)

(500.0, 0.0)

In [25]:
env.close()

In [26]:
env.action_space.sample()

0

## Test the Model

In [ ]:
# test the model

env = gym.make("CartPole-v1", render_mode="human")
model = PPO.load(model_path, env=env)

episodes = 5

for e in range(1, episodes+1):
    obs, _ = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, _, info = env.step(action)
        score += reward
    print(f"Episode #: {e}")
    print(f"Score: {score}")

env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Episode #: 1
Score: 96.0
Episode #: 2
Score: 4363.0
